In [282]:
import pandas as pd
import pandapower as pp
import pandapower.networks as ppnets
import pandapower.plotting as plt
import matplotlib.pyplot as mplt

In [283]:
def read(file_path: str, sheet_name: str):
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    return df

In [284]:
file_path = 'ac_case25.xlsx'

BUS = read(file_path, 'bus')
GEN = read(file_path, 'generator')
TR = read(file_path, 'transformer')
LINE = read(file_path, 'branch')
BASE = read(file_path, 'param')

In [285]:
net = pp.create.create_empty_network(name='case25', f_hz=60.0, sn_mva=100, add_stdtypes=True)

In [286]:
G = GEN.groupby('Bus', as_index=False).agg({
    'PG (MW)': 'sum',
    'QG (MVAR)': 'sum',
    'QMAX (MVAR)': 'sum',
    'QMIN (MVAR)': 'sum',
    'Voltage setpoint (pu)': 'first'  # 또는 'mean'
})

# print(G)

In [287]:
B = BUS.copy()

B.rename(columns={
    'Pload (MW)': 'P (MW)',
    'Qload (MVAR)': 'Q (MVAR)',
}, inplace=True)

B.loc[:,'P (MW)'] = B.loc[:,'P (MW)']*(-1) 
B.loc[:,'Q (MVAR)'] = B.loc[:,'Q (MVAR)']*(-1) 

# print(BUS)

In [288]:
for i in range(len(G)):
    B.loc[G.loc[i,'Bus']-1,'P (MW)'] += G.loc[i,'PG (MW)']

B['Vm (pu)'] = B['Vm (pu)'].astype(float)

for i in range(len(G)):
    B.loc[G.loc[i,'Bus']-1,'Vm (pu)'] = G.loc[i,'Voltage setpoint (pu)']

# print(B)

In [289]:
def mkbus(net, name,index, geodata, type_, zone, in_service, max_vm_pu, min_vm_pu, coords,vn_kv=1.0):
    bus = pp.create_bus(
        net,
        vn_kv=vn_kv,
        name=name,
        index=index,
        geodata=geodata,
        type=type_,
        zone=zone,
        in_service=in_service,
        max_vm_pu=max_vm_pu,
        min_vm_pu=min_vm_pu,
        coords=coords
    )
    return bus

In [ ]:
for i in range(len(B)):
    mkbus(net,i+1,i+1,None,'b',None,True,B.loc[i,'maxVm'],B.loc[i,'minVm'],None)

In [291]:
net.bus

,name,vn_kv,type,zone,in_service,geo,min_vm_pu,max_vm_pu
1,1,100.0,b,None,True,None,0.95,1.05
2,2,100.0,b,None,True,None,0.95,1.05
3,3,100.0,b,None,True,None,0.95,1.05
4,4,100.0,b,None,True,None,0.95,1.05
5,5,100.0,b,None,True,None,0.95,1.05
6,6,100.0,b,None,True,None,0.95,1.05
7,7,100.0,b,None,True,None,0.95,1.05
8,8,100.0,b,None,True,None,0.95,1.05
9,9,100.0,b,None,True,None,0.95,1.05
10,10,100.0,b,None,True,None,0.95,1.05


In [292]:
def mkgen(
    net,
    bus,
    index,
    p_mw, 
    max_q_mvar, 
    min_q_mvar,
    max_vm_pu, 
    min_vm_pu,
    vm_pu=1.0, 
    slack=False,
):
    pp.create.create_gen(
        net,
        bus=bus,
        index=index,
        p_mw=p_mw,
        vm_pu=vm_pu,
        max_q_mvar=max_q_mvar, 
        min_q_mvar=min_q_mvar, 
        min_vm_pu=min_vm_pu, 
        max_vm_pu=max_vm_pu, 
        slack=slack, 
    )


In [ ]:
for i in range(len(G)):
    mkgen(
        net,
        G.loc[i, 'Bus'],
        G.loc[i, 'Bus'],
        G.loc[i, 'PG (MW)'],
        G.loc[i, 'QMAX (MVAR)'],
        G.loc[i, 'QMIN (MVAR)'],
        B.loc[G.loc[i, 'Bus'] + 1, 'maxVm'],
        B.loc[G.loc[i, 'Bus'] + 1, 'minVm'],
        G.loc[i, 'Voltage setpoint (pu)'],
        B.loc[G.loc[i, 'Bus'] - 1, 'Type'] == 'Swing'
    )


In [294]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,max_vm_pu,min_vm_pu
1,None,1,172.0,1.035,NaN,-50.0,80.0,1.0,False,True,0.0,None,1.05,0.95
2,None,2,172.0,1.035,NaN,-50.0,80.0,1.0,False,True,0.0,None,1.05,0.95
7,None,7,240.0,1.025,NaN,0.0,180.0,1.0,False,True,0.0,None,1.05,0.95
13,None,13,285.3,1.020,NaN,0.0,240.0,1.0,True,True,0.0,None,1.05,0.95
14,None,14,0.0,0.980,NaN,-50.0,200.0,1.0,False,True,0.0,None,1.05,0.95
15,None,15,215.0,1.014,NaN,-50.0,110.0,1.0,False,True,0.0,None,1.05,0.95
16,None,16,155.0,1.017,NaN,-50.0,80.0,1.0,False,True,0.0,None,1.05,0.95
18,None,18,400.0,1.050,NaN,-50.0,200.0,1.0,False,True,0.0,None,1.05,0.95
21,None,21,400.0,1.050,NaN,-50.0,200.0,1.0,False,True,0.0,None,1.05,0.95
22,None,22,300.0,1.050,NaN,-60.0,96.0,1.0,False,True,0.0,None,1.05,0.95


In [295]:
def mkshunt(net, 
            bus,
            index,
            q_mvar,
            p_mw = 0):
    pp.create_shunt(net, 
                bus=bus, 
                index=index,
                q_mvar=q_mvar, 
                p_mw=p_mw)

In [296]:
mkshunt(net,14,14,35.3,)

In [297]:
net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service
14,14,None,35.3,0.0,100.0,1,1,<NA>,False,True


In [298]:
def mkload(net,
           bus,
           index,
           p_mw,
           q_mvar,
            ):

    pp.create_load(net, 
                   bus=bus, 
                   index=index,
                   p_mw=p_mw, 
                   q_mvar=q_mvar)
    

In [299]:
for i in range(len(BUS)):
    mkload(
        net,
        BUS.loc[i, 'Bus'],
        BUS.loc[i, 'Bus'],
        BUS.loc[i, 'Pload (MW)'],
        BUS.loc[i, 'Qload (MVAR)'],
    )

In [300]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
1,None,1,108.00,22.00,0.0,0.0,NaN,1.0,True,wye
2,None,2,97.00,20.00,0.0,0.0,NaN,1.0,True,wye
3,None,3,207.00,42.55,0.0,0.0,NaN,1.0,True,wye
4,None,4,74.00,15.00,0.0,0.0,NaN,1.0,True,wye
5,None,5,71.00,14.00,0.0,0.0,NaN,1.0,True,wye
6,None,6,136.00,28.00,0.0,0.0,NaN,1.0,True,wye
7,None,7,125.00,25.00,0.0,0.0,NaN,1.0,True,wye
8,None,8,136.80,28.00,0.0,0.0,NaN,1.0,True,wye
9,None,9,175.00,36.00,0.0,0.0,NaN,1.0,True,wye
10,None,10,156.00,32.00,0.0,0.0,NaN,1.0,True,wye


In [301]:
def mkimp(
        net,
        from_bus,
        to_bus,
        rft_pu,
        xft_pu,
        bf_pu,
        bt_pu,
        sn_mva = 100
        ):
    pp.create_impedance(net, 
                        from_bus=from_bus, 
                        to_bus=to_bus, 
                        rft_pu=rft_pu, 
                        xft_pu=xft_pu,
                        bf_pu=bf_pu,
                        bt_pu=bt_pu, 
                        sn_mva=sn_mva, )
    

In [302]:
for i in range(len(LINE)):
    mkimp(
        net,
        LINE.loc[i, 'From'],
        LINE.loc[i, 'To'],
        LINE.loc[i, 'R (pu)'],
        LINE.loc[i, 'X (pu)'],
        LINE.loc[i, 'B (pu)'] / 2,
        LINE.loc[i, 'B (pu)'] / 2,
        100
    )


In [303]:
net.impedance

,name,from_bus,to_bus,rft_pu,xft_pu,rtf_pu,xtf_pu,gf_pu,bf_pu,gt_pu,bt_pu,sn_mva,in_service
0,None,1,2,0.00260,0.013900,0.00260,0.013900,0.0,0.23055,0.0,0.23055,100.0,True
1,None,1,3,0.05460,0.211200,0.05460,0.211200,0.0,0.02860,0.0,0.02860,100.0,True
2,None,1,5,0.02180,0.084500,0.02180,0.084500,0.0,0.01145,0.0,0.01145,100.0,True
3,None,2,4,0.03280,0.126700,0.03280,0.126700,0.0,0.01715,0.0,0.01715,100.0,True
4,None,2,6,0.04970,0.192000,0.04970,0.192000,0.0,0.02600,0.0,0.02600,100.0,True
5,None,3,9,0.03080,0.119000,0.03080,0.119000,0.0,0.01610,0.0,0.01610,100.0,True
6,None,3,24,0.00230,0.083900,0.00230,0.083900,0.0,0.00000,0.0,0.00000,100.0,True
7,None,4,9,0.02680,0.103700,0.02680,0.103700,0.0,0.01405,0.0,0.01405,100.0,True
8,None,5,10,0.02280,0.088300,0.02280,0.088300,0.0,0.01195,0.0,0.01195,100.0,True
9,None,6,10,0.01390,0.060500,0.01390,0.060500,0.0,1.22950,0.0,1.22950,100.0,True


In [304]:
# def mktrafo(
#     net,
#     hv_bus,
#     lv_bus,
#     vn_hv_kv,
#     vn_lv_kv,
#     vkr_percent,
#     vk_percent,
#     pfe_kw,
#     i0_percent,
#     sn_mva=100,
# ):
#     pp.create_transformer_from_parameters(
#         net,
#         hv_bus=hv_bus,
#         lv_bus=lv_bus,
#         sn_mva=sn_mva,
#         vn_hv_kv=vn_hv_kv,
#         vn_lv_kv=vn_lv_kv,
#         vkr_percent=vkr_percent,
#         vk_percent=vk_percent,
#         pfe_kw=pfe_kw,
#         i0_percent=i0_percent
#     )


In [305]:
# for i in range(len(TR)):
#     mktrafo(
#         net,
#         TR.loc[i,'To'],
#         TR.loc[i,'From'],
#         110,
#         110 / TR.loc[i,'Tap'],
#         10,
#         0.5,
#         100,
#         0.1,
#         100,
#     )

In [306]:
# net.trafo

In [307]:
pp.runpp(net,numba=False)

In [308]:
Ybus = net._ppc["internal"]["Ybus"] 

In [309]:
Ybus_dense = Ybus.todense()
print(Ybus_dense)

[[ 17.01191877 -84.77385682j -13.00195029 +69.51042656j
   -1.14738183  +4.43822421j   0.          +0.j
   -2.86258665 +11.09580605j   0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j        ]
 [-13.00195029 +69.51042656j  16.18039669 -81.51491761j
    0.          +0.j          -1.91491138  +7.39692902j
    0.          +0.j          -1.26353502  +4.88126203j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j           0.          +0.j
    0.          +0.j      

In [310]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
1,1.035000,-6.528671,-64.00000,4.411650
2,1.035000,-6.578872,-75.00000,29.207867
3,0.967200,-6.511088,207.00000,42.550000
4,0.983273,-8.827168,74.00000,15.000000
5,1.029799,-8.843646,71.00000,14.000000
6,1.081480,-11.528506,136.00000,28.000000
7,1.025000,-4.526874,-115.00000,-13.586128
8,1.000722,-8.345425,136.80000,28.000000
9,0.987597,-6.781475,175.00000,36.000000
10,1.050936,-8.045438,156.00000,32.000000
